In [9]:
import torch
from torch import nn
import os

torch.set_num_threads(1)

In [10]:
import numpy as np

def save_to(base, x, y):
    np_input = x.detach().numpy()
    with open(os.path.join(base, 'dummy_in.dat'), 'wb') as f:
        np_input.tofile(f)

    np_output = y.detach().numpy()
    with open(os.path.join(base, 'dummy_out.dat'), 'wb') as f:
        np_output.tofile(f)

In [2]:
conv = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False)
)

In [3]:
x = torch.rand(10000, 1, 28, 28)

In [4]:
%%time

y = conv(x)

CPU times: user 752 ms, sys: 1.5 s, total: 2.26 s
Wall time: 204 ms


In [5]:
input_names = ["image"]
output_names = ["prediction"]
dummy_input = torch.randn(10000, 1, 28, 28)
torch.onnx.export(conv,
                  dummy_input,
                  "conv/convolution1.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [6]:
conv2 = nn.Conv2d(32, 64, 3, 1)

In [7]:
x = torch.rand(10000, 32, 26, 26)

In [8]:
%%time
y = conv2(x)
y.size()

CPU times: user 6.72 s, sys: 2.8 s, total: 9.52 s
Wall time: 834 ms


torch.Size([10000, 64, 24, 24])

In [9]:
input_names = ["image"]
output_names = ["prediction"]
dummy_input = torch.randn(10000, 32, 26, 26)
torch.onnx.export(conv2,
                  dummy_input,
                  "conv/convolution2.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [12]:
r2 = nn.ReLU()

In [13]:
%%time
y = r2(y)
y.size()

CPU times: user 508 ms, sys: 1.62 s, total: 2.13 s
Wall time: 136 ms


torch.Size([10000, 64, 24, 24])

In [18]:
input_names = ["image"]
output_names = ["prediction"]
dummy_input = torch.randn(10000, 64, 24, 24)
torch.onnx.export(r2,
                  dummy_input,
                  "relu/r2.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [15]:
mp = nn.MaxPool2d(kernel_size=2, stride=2)

In [16]:
%%time
y = mp(y)
y.size()

CPU times: user 2.6 s, sys: 500 ms, total: 3.1 s
Wall time: 229 ms


torch.Size([10000, 64, 12, 12])

In [19]:
input_names = ["image"]
output_names = ["prediction"]
dummy_input = torch.randn(10000, 64, 24, 24)
torch.onnx.export(mp,
                  dummy_input,
                  "maxpool/maxpool.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [20]:
softmax = nn.LogSoftmax(dim=1)
x = torch.rand(10000, 10)

In [21]:
%%time
y = softmax(x)
y.size()

CPU times: user 80 ms, sys: 200 ms, total: 280 ms
Wall time: 208 ms


torch.Size([10000, 10])

In [23]:
input_names = ["image"]
output_names = ["prediction"]
dummy_input = torch.randn(10000, 10)
torch.onnx.export(softmax,
                  dummy_input,
                  "Lsm/lsm.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [6]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=2, batch_first=False)
x = torch.rand(7 * 24 * 6, 64, 512)
print(encoder_layer)

TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
  )
  (linear1): Linear(in_features=512, out_features=2048, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=2048, out_features=512, bias=True)
  (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)


In [7]:
%%time

y = encoder_layer(x)
y.size()

CPU times: user 14.6 s, sys: 2.99 s, total: 17.6 s
Wall time: 17.6 s


torch.Size([1008, 64, 512])

In [11]:
save_to('attention', x, y)

In [8]:
input_names = ["series"]
output_names = ["prediction"]
dummy_input = x
torch.onnx.export(encoder_layer,
                  dummy_input,
                  "attention/encoder.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)